## NTUEE儀表板資料處理中心 - 祝您好孕
##### 作者：NTUEE儀表板資料組

#### 引入必要套件與函數

In [190]:
%%capture 
!pip3 install pandas
!pip3 install geocoder
!pip3 install geopandas
!pip3 install shapely
!pip3 install openpyxl

##### 安裝套件、函數宣告


In [191]:
%%capture
import json
import requests
import pandas as pd
import geocoder
import geopandas as gpd
from shapely.geometry import Point, Polygon
from IPython.display import display, HTML
## 從data.taipei抓取資料
def get_datataipei_api(rid):
    url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire"""
    response = requests.request("GET", url)
    data_dict = response.json()
    count = data_dict['result']['count']
    res = list()
    offset_count = int(count/1000)
    for i in range(offset_count+1):
        i = i* 1000
        url = f"""https://data.taipei/api/v1/dataset/{rid}?scope=resourceAquire&offset={i}&limit=1000"""
        response = requests.request("GET", url)
        get_json = response.json()
        res.extend(get_json['result']['results'])
    return pd.DataFrame(res)
## 用geocoder取得TWD97座標
def 新增經緯度TWD97座標行以及轉GeoDataFrame(資料, 地址欄位名稱):
	資料['geometry'] = 資料[地址欄位名稱].apply(
		lambda x: (
			lambda a: Point(a.json['lng'], a.json['lat']) if a and a.json else None
		)(geocoder.arcgis(x))
	)
	# 沒查到經緯度的話...郭囉 誰叫你選那麼奇怪的址
	資料.dropna(subset=['geometry'], inplace=True)
	資料 = gpd.GeoDataFrame(
		資料,
		geometry='geometry', 
		crs='EPSG:4326'
	)
	# 資料 = 資料.to_crs('EPSG:3826') # TWD97
	return 資料
## 印資料
def 印資料(資料):
	print("這份資料總共有", 資料.shape[0], "筆！")
	display(資料)

In [203]:
GCoder = geocoder.arcgis('臺北市大同區中山捷運站').json
print(GCoder)

{'address': '中山捷運站', 'bbox': {'northeast': [25.05758, 121.52536], 'southwest': [25.04758, 121.51536]}, 'confidence': 7, 'lat': 25.052580000000034, 'lng': 121.5203600000001, 'ok': True, 'quality': 'POI', 'raw': {'name': '中山捷運站', 'extent': {'xmin': 121.51536, 'ymin': 25.04758, 'xmax': 121.52536, 'ymax': 25.05758}, 'feature': {'geometry': {'x': 121.5203600000001, 'y': 25.052580000000034}, 'attributes': {'Score': 83.91, 'Addr_Type': 'POI'}}}, 'score': 83.91, 'status': 'OK'}


### 祝您好孕

##### 資料庫（不斷更新！）


In [382]:
祝您好孕資料庫 = {
    "112上半年婦幼安全警示地點": "016bf772-71bd-4500-aba2-22387a1d0ae7",
	"公辦幼兒園班級數": "./公辦幼兒園.xlsx",
	"哺集乳室": "4350117f-8697-4903-8a49-a7a59f30eeb1",
	"各區人口": "edf9a589-7095-4f18-995f-f8657c0d8c1a",
	"支出綜合資料": "./支出綜合資料.csv",
	"物價指數": "./物價指數.csv",
	"家庭組織型態": "./家庭組織型態.csv",
	"薪資": "./薪資.csv",
	"所有幼兒園": "./all_kindergarten.json",
	"核心物價指數": "e488271c-827f-469d-ae2b-758e38caddcc",
	"所得物價指數": "d2bdc07c-3362-4a92-b042-69e3420c3f94",
	"生育率": "./生育率.csv"
}

In [242]:
各區人口 = get_datataipei_api(祝您好孕資料庫["各區人口"])

##### 點子
1. 婦幼安全警示區分佈圖
2. 幼兒園分佈圖

##### 婦幼安全警示區分佈圖

In [200]:
# 取得資料
婦幼安全警示區資料 = get_datataipei_api(祝您好孕資料庫["112上半年婦幼安全警示地點"])
婦幼安全警示區資料.drop('_id', axis=1, inplace=True)
婦幼安全警示區資料.drop('_importdate', axis=1, inplace=True)
印資料(婦幼安全警示區資料)


這份資料總共有 20 筆！


,編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註
0,1\t臺北市大同區捷運中山站\t大同分局\t103\t續列
1,2\t臺北市萬華區峨嵋街、漢中街口(捷運西門站)\t萬華分局\t108\t續列
2,3\t臺北市萬華區西園路、和平西路口(捷運龍山寺站)\t萬華分局\t108\t續列
3,4\t臺北市中山區捷運松江南京站\t中山分局\t104\t續列
4,5\t臺北市中山區捷運中山國小站\t中山分局\t104\t續列
5,6\t臺北市大安區捷運忠孝復興站\t大安分局\t106\t續列
6,7\t臺北市中正區捷運臺北車站\t中正第一分局\t100\t續列
7,8\t臺北市中正區二二八公園\t中正第一分局\t100\t續列
8,9\t臺北市中正區捷運古亭站\t中正第二分局\t100\t續列
9,10\t臺北市松山區新東街9巷\t松山分局\t105\t續列


In [201]:
# 整理一下資料
婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][10] += 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'][11]
婦幼安全警示區資料.drop(11, inplace=True)
婦幼安全警示區資料.reset_index(drop=True, inplace=True)
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[1]
婦幼安全警示區資料['管轄分局'] = 婦幼安全警示區資料['編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註'].str.split('\t').str[2]
婦幼安全警示區資料.drop('編號\t地點位置\t管轄分局\t郵遞區號（3碼）\t備註', axis=1, inplace=True)
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['地點位置'].str.replace('捷運', '')
婦幼安全警示區資料['地點位置'] = 婦幼安全警示區資料['地點位置'].str.replace('站', '捷運站')
印資料(婦幼安全警示區資料)

這份資料總共有 19 筆！


,地點位置,管轄分局
0,臺北市大同區中山站,大同分局
1,臺北市萬華區峨嵋街、漢中街口(西門站),萬華分局
2,臺北市萬華區西園路、和平西路口(龍山寺站),萬華分局
3,臺北市中山區松江南京站,中山分局
4,臺北市中山區中山國小站,中山分局
5,臺北市大安區忠孝復興站,大安分局
6,臺北市中正區臺北車站,中正第一分局
7,臺北市中正區二二八公園,中正第一分局
8,臺北市中正區古亭站,中正第二分局
9,臺北市松山區新東街9巷,松山分局


這份資料總共有 19 筆！


,地點位置,管轄分局
0,臺北市大同區中山捷運站,大同分局
1,臺北市萬華區峨嵋街、漢中街口(西門捷運站),萬華分局
2,臺北市萬華區西園路、和平西路口(龍山寺捷運站),萬華分局
3,臺北市中山區松江南京捷運站,中山分局
4,臺北市中山區中山國小捷運站,中山分局
5,臺北市大安區忠孝復興捷運站,大安分局
6,臺北市中正區臺北車捷運站,中正第一分局
7,臺北市中正區二二八公園,中正第一分局
8,臺北市中正區古亭捷運站,中正第二分局
9,臺北市松山區新東街9巷,松山分局


In [202]:
# 找座標，並轉成GeoDataFrame
婦幼安全警示區資料GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(婦幼安全警示區資料, '地點位置')
印資料(婦幼安全警示區資料GDF)

這份資料總共有 18 筆！


,地點位置,管轄分局,geometry
0,臺北市大同區中山捷運站,大同分局,POINT (121.52036 25.05258)
1,臺北市萬華區峨嵋街、漢中街口(西門捷運站),萬華分局,POINT (121.50696 25.04150)
2,臺北市萬華區西園路、和平西路口(龍山寺捷運站),萬華分局,POINT (121.49413 25.03575)
3,臺北市中山區松江南京捷運站,中山分局,POINT (121.53274 25.05269)
4,臺北市中山區中山國小捷運站,中山分局,POINT (121.52641 25.06276)
5,臺北市大安區忠孝復興捷運站,大安分局,POINT (121.54335 25.04145)
6,臺北市中正區臺北車捷運站,中正第一分局,POINT (121.51856 25.03247)
7,臺北市中正區二二八公園,中正第一分局,POINT (121.51856 25.03247)
8,臺北市中正區古亭捷運站,中正第二分局,POINT (121.52275 25.02713)
9,臺北市松山區新東街9巷,松山分局,POINT (121.56510 25.05785)


In [168]:
# 儲存成GeoJSON
婦幼安全警示區資料GDF.to_file(
    './112上半年婦幼安全警示區地點.geojson',
    driver='GeoJSON'
)

##### 幼兒園分佈圖

In [169]:
# 取得資料
公辦幼兒園班級數 = pd.read_excel(祝您好孕資料庫["公辦幼兒園班級數"])
印資料(公辦幼兒園班級數)

這份資料總共有 147 筆！


,區,校園名稱,3-5\n歲班,2 歲專班,總班\n數,3-5歲班缺額,2 歲專班缺額,總缺\n額
0,松山區,松山附幼,5,2.0,7,13,31.0,44
1,松山區,**西松附幼,7,2.0,9,49,30.0,79
2,松山區,**敦化附幼,4,NaN,4,64,NaN,64
3,松山區,民權附幼,6,2.0,8,47,32.0,79
4,松山區,民族附幼,6,NaN,6,61,NaN,61
...,...,...,...,...,...,...,...,...
142,北投區,義方附幼,3,1.0,4,3,1.0,4
143,北投區,立農附幼,4,NaN,4,4,NaN,4
144,北投區,明德附幼,4,2.0,6,4,2.0,6
145,北投區,文化附幼,2,NaN,2,2,NaN,2


In [170]:
# 整理一下資料
公辦幼兒園班級數.fillna(0, inplace=True)
公辦幼兒園班級數.columns = 公辦幼兒園班級數.columns.str.replace('\n', '')
公辦幼兒園班級數.columns = 公辦幼兒園班級數.columns.str.replace(' ', '')
公辦幼兒園班級數['校園名稱'] = 公辦幼兒園班級數['校園名稱'].str.replace('*', '')
公辦幼兒園班級數['2歲專班'] = 公辦幼兒園班級數['2歲專班'].astype(int)
公辦幼兒園班級數['2歲專班缺額'] = 公辦幼兒園班級數['2歲專班缺額'].astype(int)
公辦幼兒園班級數['3-5歲核定人數'] = 公辦幼兒園班級數['3-5歲班'].astype(int) * 30
公辦幼兒園班級數['2歲核定人數'] = 公辦幼兒園班級數['2歲專班'].astype(int) * 16
公辦幼兒園班級數['核定人數'] = 公辦幼兒園班級數['3-5歲核定人數'] + 公辦幼兒園班級數['2歲核定人數']
公辦幼兒園班級數['實際就讀人數'] = 公辦幼兒園班級數['核定人數'] - 公辦幼兒園班級數['總缺額']
公辦幼兒園就讀人數 = 公辦幼兒園班級數.groupby('區').sum()
公辦幼兒園就讀人數.drop('校園名稱', axis=1, inplace=True)
公辦幼兒園班級數['校園名稱'] = "臺北市" + 公辦幼兒園班級數['區'] + 公辦幼兒園班級數['校園名稱'].str.replace('附幼', '國小附設幼兒園')
公辦幼兒園班級數.drop('區', axis=1, inplace=True)
印資料(公辦幼兒園班級數)

這份資料總共有 147 筆！


,校園名稱,3-5歲班,2歲專班,總班數,3-5歲班缺額,2歲專班缺額,總缺額,3-5歲核定人數,2歲核定人數,核定人數,實際就讀人數
0,臺北市松山區松山國小附設幼兒園,5,2,7,13,31,44,150,32,182,138
1,臺北市松山區西松國小附設幼兒園,7,2,9,49,30,79,210,32,242,163
2,臺北市松山區敦化國小附設幼兒園,4,0,4,64,0,64,120,0,120,56
3,臺北市松山區民權國小附設幼兒園,6,2,8,47,32,79,180,32,212,133
4,臺北市松山區民族國小附設幼兒園,6,0,6,61,0,61,180,0,180,119
...,...,...,...,...,...,...,...,...,...,...,...
142,臺北市北投區義方國小附設幼兒園,3,1,4,3,1,4,90,16,106,102
143,臺北市北投區立農國小附設幼兒園,4,0,4,4,0,4,120,0,120,116
144,臺北市北投區明德國小附設幼兒園,4,2,6,4,2,6,120,32,152,146
145,臺北市北投區文化國小附設幼兒園,2,0,2,2,0,2,60,0,60,58


In [171]:
公辦幼兒園就讀人數['2歲該區人數'] = {}
公辦幼兒園就讀人數['3-5歲該區人數'] = {}
for i in range(公辦幼兒園就讀人數.shape[0]):
    公辦幼兒園就讀人數['2歲該區人數'][i] = 各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['2歲數量'].values[0]
    公辦幼兒園就讀人數['3-5歲該區人數'][i] = int(各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['3歲數量'].values[0]) \
    + int(各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['4歲數量'].values[0]) \
    + int(各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['5歲數量'].values[0])
公辦幼兒園就讀人數['2歲就讀率'] = (公辦幼兒園就讀人數['2歲核定人數'].astype(int) - 公辦幼兒園就讀人數['2歲專班缺額'].astype(int)) / 公辦幼兒園就讀人數['2歲該區人數'].astype(int) * 100
公辦幼兒園就讀人數['3-5歲就讀率'] = (公辦幼兒園就讀人數['3-5歲核定人數'].astype(int) - 公辦幼兒園就讀人數['3-5歲班缺額'].astype(int)) / 公辦幼兒園就讀人數['3-5歲該區人數'].astype(int) * 100
印資料(公辦幼兒園就讀人數)
公辦幼兒園就讀人數.to_json('./公辦幼兒園就讀人數.json', orient='index', force_ascii=False)

這份資料總共有 12 筆！


/var/folders/rc/m49gmyzd7q7_y4d3f8h494h00000gn/T/ipykernel_13438/4206700593.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  公辦幼兒園就讀人數['2歲該區人數'][i] = 各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['2歲數量'].values[0]
/var/folders/rc/m49gmyzd7q7_y4d3f8h494h00000gn/T/ipykernel_13438/4206700593.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  公辦幼兒園就讀人數['3-5歲該區人數'][i] = int(各區人口[(各區人口['區域別'] == 公辦幼兒園就讀人數.index[i]) & (各區人口['年份'] == str(112)) & (各區人口['月份'] == str(6)) & (各區人口['性別'] == "計")]['3歲數量'].values[0]) \
/var/folders/rc/m49gmyzd7q7_y4d3f8h494h0000

,3-5歲班,2歲專班,總班數,3-5歲班缺額,2歲專班缺額,總缺額,3-5歲核定人數,2歲核定人數,核定人數,實際就讀人數,2歲該區人數,3-5歲該區人數,2歲就讀率,3-5歲就讀率
區,,,,,,,,,,,,,,
中山區,57,17,74,419,270,689,1710,272,1982,1293,1404,4893,0.142450,26.384631
中正區,33,4,37,341,61,402,990,64,1054,652,1109,4074,0.270514,15.930290
信義區,48,12,60,360,188,548,1440,192,1632,1084,1345,4535,0.297398,23.814774
內湖區,65,17,82,156,61,217,1950,272,2222,2005,2159,7317,9.773043,24.518245
北投區,54,12,66,54,12,66,1620,192,1812,1746,1621,5802,11.104257,26.990693
南港區,28,9,37,198,141,339,840,144,984,645,830,2916,0.361446,22.016461
士林區,64,18,82,64,18,82,1920,288,2208,2126,1676,6068,16.109785,30.586684
大同區,42,12,54,266,188,454,1260,192,1452,998,799,2872,0.500626,34.610028
大安區,52,4,56,529,61,590,1560,64,1624,1034,2035,7309,0.147420,14.105897


In [172]:
# 找座標，並轉成GeoDataFrame
公辦幼兒園班級數GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(公辦幼兒園班級數, '校園名稱')
印資料(公辦幼兒園班級數GDF)

KeyboardInterrupt: 

In [ ]:
公辦幼兒園班級數GDF.to_file(
    './112學年度公辦幼兒園班級數.geojson',
    driver='GeoJSON'
)

##### 哺集乳室

In [204]:
# 取得資料
哺集乳室資料 = get_datataipei_api(祝您好孕資料庫["哺集乳室"])
哺集乳室資料.drop('_id', axis=1, inplace=True)
哺集乳室資料.drop('_importdate', axis=1, inplace=True)
印資料(哺集乳室資料)

這份資料總共有 538 筆！


,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,優良哺集乳室認證效期,輪椅使用,貼心小提醒
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;消毒器具;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設置意...,無,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;母乳哺育宣導資訊;設置意見回饋箱,112年8月,無,使用者請於開放時間向本署秘書室事務課洽借鑰匙，並填寫哺(集)乳室鑰匙借用登記表、使用登記統計...
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊,無,可提供輪椅媽媽使用,哺集乳室資訊依各場所實際公告內容為主
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室；哺集乳室資訊依各場所實際公告內容為主
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;進行滿意度調查,112年8月,無,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;置物空間;母乳哺育宣導資訊,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,飲水服務;尿布台;母乳專用冰箱;免費衛生用品;置物空間;嬰兒車停放空間;母乳哺育宣導資訊;設...,112年8月,可提供輪椅媽媽使用,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,靠背椅;有蓋垃圾桶;電源設備 （插座或延長線）;可由內部上鎖之門;緊急求救設備 （求救鈴或電...,以室內設備為主,無,以室內設備為主,自行前往哺集乳室，並登記；哺集乳室資訊依各場所實際公告內容為主


In [185]:
# Custom function to process each row
哺集乳室資料深拷貝 = 哺集乳室資料.copy(deep=True)
def 處理基本資料字串(列, 獨特元素列表):
    if "以室內設備為主" in 列 or "以上皆無提供" in 列:
        return "無"
    設備字典 = {}
    for 元素 in 獨特元素列表:
        元素 = 元素.strip()
        if 元素 and 元素 != "以室內設備為主" and 元素 != "以上皆無提供" and 元素 != "\u3000music" and 元素 != "music":
            設備字典[元素] = "有" if 元素 in 列 else "無"
        if 元素 == '毒器具':
            設備字典['消毒器具'] = "有"
            設備字典.pop(元素)
        if 元素 == '母乳專用冰箱消毒器具':
            設備字典['消毒器具'] = "有"
            設備字典['母乳專用冰箱'] = "有"
            設備字典.pop(元素)
        if 元素 in {'設置意見回饋', '設置滿意度調查表', '飲水服務在1樓大廳，有意見調查表 （回饋箱置於1樓大廳）', '箱進行滿意度調查', '滿意度調查', '進行滿意度調查', '設置意見回饋箱'}:
            設備字典['設置意見回饋或滿意度調查'] = "有"
            設備字典.pop(元素)
        if 元素 == '其他（多功能溫奶器）':
            設備字典['多功能溫奶器'] = "有"
            設備字典.pop(元素)
        if 元素 == '消毒器具置物空間':
            設備字典['消毒器具'] = "有"
            設備字典['置物空間'] = "有"
            設備字典.pop(元素)
        if 元素 == '母乳哺育宣導資訊箱':
            設備字典.pop(元素)
    return 設備字典
全部的元素 = set(';'.join(哺集乳室資料['友善設備或服務']).split(';'))
基本設備字典 = {
    '靠背椅': '有',
    '有蓋垃圾桶': '有',
    '電源設備 （插座或延長線）': '有',
    '可由內部上鎖之門': '有',
    '緊急求救設備 （求救鈴或電話等）': '有',
    '洗手設施 （洗手台或乾洗手液）': '有'
}
哺集乳室資料深拷貝['友善設備或服務'] = 哺集乳室資料深拷貝['友善設備或服務'].apply(lambda x: json.dumps(處理基本資料字串(x.split(';'), 全部的元素), ensure_ascii=False))
哺集乳室資料深拷貝['基本設備'] = 哺集乳室資料深拷貝['基本設備'].apply(lambda x: json.dumps(基本設備字典, ensure_ascii=False))
哺集乳室資料深拷貝['可提供輪椅媽媽使用'] = 哺集乳室資料深拷貝['輪椅使用'].apply(lambda x: "是" if x == "可提供輪椅媽媽使用" else "否")
哺集乳室資料深拷貝['營業中'] = 哺集乳室資料深拷貝['開放時間'].apply(lambda x: "是")
哺集乳室資料深拷貝.drop('貼心小提醒', axis=1, inplace=True)
哺集乳室資料深拷貝.drop('優良哺集乳室認證效期', axis=1, inplace=True)
哺集乳室資料深拷貝.drop('輪椅使用', axis=1, inplace=True)



In [186]:
display(哺集乳室資料深拷貝)

,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,可提供輪椅媽媽使用,營業中
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",否,是
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",否,是
...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","""無""",否,是
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","""無""",否,是


In [187]:
# 找座標，並轉成GeoDataFrame
哺集乳室GDF = 新增經緯度TWD97座標行以及轉GeoDataFrame(哺集乳室資料深拷貝, '地址')
印資料(哺集乳室GDF)

這份資料總共有 538 筆！


,行政區,機構名稱,地址,電話,開放時間,位置指引,基本設備,友善設備或服務,可提供輪椅媽媽使用,營業中,geometry
0,松山區,臺北市松山區健康服務中心,臺北市松山區八德路4段692號2樓,02-27671757轉6062,週一至週五08:30~17:30,松山區行政大樓2樓服務台旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是,POINT (121.57717 25.04985)
1,松山區,內政部營建署,臺北市松山區八德路2段342號1樓,02-87712437,週一至週五08:00~17:00,101室,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",否,是,POINT (121.54448 25.04759)
2,松山區,交通部運輸研究所,臺北市松山區敦化北路240號,02-23496734轉6733,每日24小時開放,地下1樓國際會議廳後方,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是,POINT (121.54949 25.06007)
3,松山區,交通部公路總局臺北市區監理所,臺北市松山區八德路4段21號,02-27630155,週一至週五08:30~18:00,2樓電梯旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是,POINT (121.55902 25.04845)
4,松山區,臺北自來水事業處東區營業分處,臺北市松山區光復北路266號,02-87703362,週一至週五08:30~17:30,1樓3號服務櫃台正對面,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",否,是,POINT (121.55470 25.05918)
...,...,...,...,...,...,...,...,...,...,...,...
533,北投區,臺北市政府消防局第四大隊陽明山分隊,臺北市北投區勝利路1號,02-28616119,每日24小時開放,1樓值班室旁,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是,POINT (121.50253 25.13289)
534,北投區,臺北市政府環境保護局北投垃圾焚化廠,臺北市北投區洲美街271號,02-28360050,週一至週五09:00~17:00,1樓主題館,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","""無""",否,是,POINT (121.49871 25.10918)
535,北投區,陽明山國家公園管理處陽明書屋,臺北市北投區中興路12號,02-28611444,週一至週五09:00~17:00,遊客中心,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","{""多功能溫奶器"": ""有"", ""設置意見回饋或滿意度調查"": ""有"", ""置物空間"": ""...",是,是,POINT (121.53941 25.16163)
536,北投區,家樂福北投店,臺北市北投區中和街366號,0963-079728,週一至週日08:00~23:00,地下1樓,"{""靠背椅"": ""有"", ""有蓋垃圾桶"": ""有"", ""電源設備 （插座或延長線）"": ""有...","""無""",否,是,POINT (121.49945 25.14169)


In [ ]:
哺集乳室GDF.to_file(
    './哺集乳室.geojson',
    driver='GeoJSON'
)

##### 經濟

In [392]:
支出綜合資料 = pd.read_csv(祝您好孕資料庫["支出綜合資料"], encoding='big5').dropna()
印資料(支出綜合資料)
物價指數 = pd.read_csv(祝您好孕資料庫["物價指數"], encoding='big5').dropna()
印資料(物價指數)
家庭組織型態 = pd.read_csv(祝您好孕資料庫["家庭組織型態"], encoding='big5').dropna()
印資料(家庭組織型態)
薪資 = pd.read_csv(祝您好孕資料庫["薪資"], encoding='big5').dropna()
印資料(薪資)


這份資料總共有 594 筆！


,統計期,統計項,總平均,單人,夫婦,單親,核心,祖孫,三代,其他
0,101年,家庭戶數(戶),1007687.00,60043,173934,105456,456066,11146,119453,81589
1,101年,每戶人數(人),3.21,1.00,2.00,2.49,3.71,2.59,5.04,3.00
2,101年,每戶成年人數(人),2.64,1.00,2.00,2.18,2.91,1.77,3.75,2.81
3,101年,每戶就業人數(人),1.39,0.42,0.53,1.25,1.76,0.50,1.91,1.39
4,101年,每戶所得收入者人數(人),1.71,1.00,1.28,1.52,1.92,1.23,2.10,1.72
...,...,...,...,...,...,...,...,...,...,...
589,111年,12.什項消費,52378.00,25927,45024,37450,64920,30871,82394,54011
590,111年,可支配所得(元),1444264.00,774607,1231880,1177973,1799675,816798,1996559,1388186
591,111年,消費支出(元),1096896.00,609128,961802,890553,1340486,820287,1539042,1061613
592,111年,儲蓄(元),347368.00,165478,270078,287421,459189,-3489,457517,326572


這份資料總共有 1200 筆！


,年別,基本分類,原始值[統計數值],年增率[%]
0,87年,總指數,79.77,-
1,87年,一.食物類,60.59,-
2,87年,(一)穀類及其製品,63.62,-
3,87年,(二)肉類,53.94,-
4,87年,(三)肉類製品,66.38,-
...,...,...,...,...
1195,111年,(二)美容及衛生用品,103.85,3.85
1196,111年,(三)個人隨身用品,101.63,1.63
1197,111年,(四)個人照顧服務費,103.60,3.6
1198,111年,(三)理容服務費,102.90,2.9


這份資料總共有 12 筆！


,統計期,主要家計負責人性別比率/總計[%],主要家計負責人性別比率/男[%],主要家計負責人性別比率/女[%],家庭組織型態結構比率/單人[%],家庭組織型態結構比率/夫婦[%],家庭組織型態結構比率/單親[%],家庭組織型態結構比率/核心[%],家庭組織型態結構比率/祖孫[%],家庭組織型態結構比率/三代[%],家庭組織型態結構比率/其他[%],與子女同住者比率[%]
0,99年,100.0,70.86,29.14,7.69,16.45,8.46,47.94,0.60,10.90,7.95,67.31
1,100年,100.0,69.95,30.05,6.00,16.90,8.81,46.34,1.16,12.45,8.34,67.60
2,101年,100.0,68.98,31.02,5.96,17.26,10.47,45.26,1.11,11.85,8.10,67.58
3,102年,100.0,69.30,30.70,6.95,17.84,10.49,43.74,0.60,11.51,8.88,65.74
4,103年,100.0,70.92,29.08,6.19,19.17,9.50,45.09,0.85,11.14,8.05,65.74
5,104年,100.0,67.96,32.04,7.41,20.33,10.19,39.91,1.15,11.71,9.30,61.80
6,105年,100.0,68.69,31.31,6.97,22.16,10.16,42.53,1.05,9.54,7.58,62.23
7,106年,100.0,70.36,29.64,5.39,21.11,9.84,42.47,1.24,10.97,8.99,63.27
8,107年,100.0,67.72,32.28,6.56,15.93,9.62,46.08,1.45,11.01,9.36,66.70
9,108年,100.0,69.91,30.09,7.66,15.79,9.14,45.97,0.75,10.79,9.90,65.90


這份資料總共有 13 筆！


,統計期,每戶平均可支配所得[元],房價所得比[第4季][倍],每人每月經常性薪資/總計[元],每人每月經常性薪資/男[元],每人每月經常性薪資/女[元],經常性薪資中位數/總計[萬元],經常性薪資中位數/男[萬元],經常性薪資中位數/女[萬元],每月基本工資[元],每小時基本工資[元],經濟成長率[%]
0,99年,1298640,12.89,36233.0,39330.0,32526.0,-,-,-,17280.0,95.0,10.25
1,100年,1251519,13.72,36735.0,39843.0,33036.0,-,-,-,17880.0,98.0,3.67
2,101年,1278278,14.80,37193.0,40228.0,33597.0,3.7,4.0,3.3,18780.0,103.0,2.22
3,102年,1279195,15.01,37552.0,40601.0,33958.0,3.7,4.0,3.4,19047.0,109.0,2.48
4,103年,1292604,15.73,38218.0,41153.0,34799.0,3.8,4.1,3.5,19273.0,115.0,4.72
5,104年,1314031,15.75,38712.0,41589.0,35390.0,3.9,4.1,3.6,20008.0,120.0,1.47
6,105年,1320834,15.18,39213.0,41962.0,36050.0,3.9,4.1,3.6,20008.0,126.0,2.17
7,106年,1344538,14.99,39928.0,42654.0,36801.0,4.0,4.2,3.7,21009.0,133.0,3.31
8,107年,1379305,13.86,40959.0,43673.0,37849.0,4.1,4.3,3.8,22000.0,140.0,2.79
9,108年,1422400,13.94,41776.0,44586.0,38630.0,4.1,4.4,3.9,23100.0,150.0,3.06


In [393]:
extract = [54 * i + j for i in range(11) for j in [0, 5, 20, 27, 28, 29, 30, 31, 34, 35, 36, 41, 42, 47, 48, 49, 50, 51, 52]]
支出綜合資料 = 支出綜合資料.iloc[extract]
支出綜合資料 = 支出綜合資料.loc[:, ['統計期','統計項', '夫婦','核心']]
支出綜合資料.reset_index(drop=True, inplace=True)

印資料(支出綜合資料)


這份資料總共有 209 筆！


,統計期,統計項,夫婦,核心
0,101年,家庭戶數(戶),173934,456066
1,101年,所得收入總計(元),1220907,1885299
2,101年,二、按支出別分/(一)非消費支出(元),205345,379257
3,101年,二、按支出別分/(二)消費支出(元),768944,1117962
4,101年,1.食品及非酒精飲料,109857,166781
...,...,...,...,...
204,111年,11.餐廳及旅館,121513,202378
205,111年,12.什項消費,45024,64920
206,111年,可支配所得(元),1231880,1799675
207,111年,消費支出(元),961802,1340486


In [394]:
支出綜合資料深拷貝 = 支出綜合資料.copy(deep=True)

In [395]:
# 取得資料
核心物價指數資料 = get_datataipei_api(祝您好孕資料庫["核心物價指數"])
所得物價指數資料 = get_datataipei_api(祝您好孕資料庫["所得物價指數"])
核心物價指數資料.drop('_id', axis=1, inplace=True)
所得物價指數資料.drop('_id', axis=1, inplace=True)
核心物價指數資料.drop('_importdate', axis=1, inplace=True)
所得物價指數資料.drop('_importdate', axis=1, inplace=True)
印資料(核心物價指數資料)
印資料(所得物價指數資料)
核心物價資料 = 核心物價指數資料.loc[核心物價指數資料["特殊分類"] == "總指數(不含蔬果及能源)【即核心物價】"]
核心物價資料 = 核心物價資料.reset_index(drop=True)
印資料(核心物價資料)

這份資料總共有 75 筆！


,年別,特殊分類,原始值[統計數值],年增率[％]
0,87年,總指數(不含食物),87.99,-
1,87年,總指數(不含蔬菜水果),81.27,-
2,87年,總指數(不含蔬果及能源)【即核心物價】,81.39,-
3,88年,總指數(不含食物),88.49,0.57
4,88年,總指數(不含蔬菜水果),82.14,1.07
...,...,...,...,...
70,110年,總指數(不含蔬菜水果),100,1.22
71,110年,總指數(不含蔬果及能源)【即核心物價】,100,0.84
72,111年,總指數(不含食物),101.91,1.91
73,111年,總指數(不含蔬菜水果),102.7,2.7


這份資料總共有 320 筆！


,年別,所得層級別,基本分類,原始值[統計數值],年增率[％]
0,102年,全體家庭,總指數,93.53,0.82
1,102年,全體家庭,一.食物類,83.05,1.07
2,102年,全體家庭,二.衣著類,94.83,-1.33
3,102年,全體家庭,三.居住類,96.61,1.01
4,102年,全體家庭,四.交通及通訊類,104.01,0.57
...,...,...,...,...,...
315,111年,可支配所得最高20%家庭,三.居住類,101.86,1.86
316,111年,可支配所得最高20%家庭,四.交通及通訊類,102.32,2.32
317,111年,可支配所得最高20%家庭,五.醫藥保健類,100.58,0.58
318,111年,可支配所得最高20%家庭,六.教養娛樂類,103.66,3.66


這份資料總共有 25 筆！


,年別,特殊分類,原始值[統計數值],年增率[％]
0,87年,總指數(不含蔬果及能源)【即核心物價】,81.39,-
1,88年,總指數(不含蔬果及能源)【即核心物價】,82.4,1.24
2,89年,總指數(不含蔬果及能源)【即核心物價】,83.45,1.27
3,90年,總指數(不含蔬果及能源)【即核心物價】,83.57,0.14
4,91年,總指數(不含蔬果及能源)【即核心物價】,83.8,0.28
5,92年,總指數(不含蔬果及能源)【即核心物價】,83.65,-0.18
6,93年,總指數(不含蔬果及能源)【即核心物價】,84.93,1.53
7,94年,總指數(不含蔬果及能源)【即核心物價】,85.86,1.1
8,95年,總指數(不含蔬果及能源)【即核心物價】,86.23,0.43
9,96年,總指數(不含蔬果及能源)【即核心物價】,87.69,1.69


In [396]:
# Insert the new row below each row with 統計項 being "儲蓄(元)"
合併資料 = pd.DataFrame()
for index in range(11):
    new_row_data = {'統計期': 支出綜合資料深拷貝.loc[index * 19 + 18]['統計期'], '統計項': '實質可支配所得', 
        '夫婦': int(float(支出綜合資料深拷貝.loc[index * 19 + 16]['夫婦']) / float(核心物價資料.loc[核心物價資料['年別'] == 支出綜合資料深拷貝.iloc[index * 19 + 1]['統計期']]['原始值[統計數值]'].iloc[0]) * 100) ,  
        '核心': int(float(支出綜合資料深拷貝.loc[index * 19 + 16]['核心']) / float(核心物價資料.loc[核心物價資料['年別'] == 支出綜合資料深拷貝.iloc[index * 19 + 1]['統計期']]['原始值[統計數值]'].iloc[0]) * 100)}
    合併資料 = pd.concat([合併資料, 支出綜合資料深拷貝.iloc[index * 19 :index * 19 + 19]]).reset_index(drop=True)
    合併資料 = pd.concat([合併資料, pd.DataFrame([new_row_data])]).reset_index(drop=True)
    if index == 10:
        break
    ratio_data = {'統計期': 支出綜合資料深拷貝.loc[index * 19 + 18]['統計期'], '統計項': '家庭組織型態結構比率',
        '夫婦': 家庭組織型態.loc[家庭組織型態['統計期'] == 支出綜合資料深拷貝.iloc[index * 19 + 1]['統計期']]['家庭組織型態結構比率/夫婦[%]'].iloc[0],
        '核心': 家庭組織型態.loc[家庭組織型態['統計期'] == 支出綜合資料深拷貝.iloc[index * 19 + 1]['統計期']]['家庭組織型態結構比率/核心[%]'].iloc[0]}
    合併資料 = pd.concat([合併資料, pd.DataFrame([ratio_data])]).reset_index(drop=True)

印資料(合併資料)
合併資料.to_json('./家庭所得消費資料.json', orient='index', force_ascii=False)

這份資料總共有 230 筆！


,統計期,統計項,夫婦,核心
0,101年,家庭戶數(戶),173934,456066
1,101年,所得收入總計(元),1220907,1885299
2,101年,二、按支出別分/(一)非消費支出(元),205345,379257
3,101年,二、按支出別分/(二)消費支出(元),768944,1117962
4,101年,1.食品及非酒精飲料,109857,166781
...,...,...,...,...
225,111年,12.什項消費,45024,64920
226,111年,可支配所得(元),1231880,1799675
227,111年,消費支出(元),961802,1340486
228,111年,儲蓄(元),270078,459189


In [397]:
所得物價指數資料 = 所得物價指數資料.loc[所得物價指數資料["基本分類"] == "總指數"].reset_index(drop=True)
印資料(所得物價指數資料)

這份資料總共有 40 筆！


,年別,所得層級別,基本分類,原始值[統計數值],年增率[％]
0,102年,全體家庭,總指數,93.53,0.82
1,102年,可支配所得最低20%家庭,總指數,92.39,-
2,102年,可支配所得中間60%家庭,總指數,93.42,-
3,102年,可支配所得最高20%家庭,總指數,93.98,-
4,103年,全體家庭,總指數,95.1,1.68
5,103年,可支配所得最低20%家庭,總指數,94.04,1.79
6,103年,可支配所得中間60%家庭,總指數,95.04,1.73
7,103年,可支配所得最高20%家庭,總指數,95.6,1.72
8,104年,全體家庭,總指數,95.1,0
9,104年,可支配所得最低20%家庭,總指數,94.45,0.44


In [398]:
生育率 = pd.read_csv(祝您好孕資料庫["生育率"], encoding='big5').dropna()
生育率 = 生育率.loc[37:].reset_index(drop=True)
生育率['全體家庭CPI'] = 所得物價指數資料.loc[所得物價指數資料['所得層級別'] == '全體家庭']['原始值[統計數值]'].reset_index(drop=True)
生育率['可支配所得最低20%家庭CPI'] = 所得物價指數資料.loc[所得物價指數資料['所得層級別'] == '可支配所得最低20%家庭']['原始值[統計數值]'].reset_index(drop=True)
生育率['可支配所得中間60%家庭CPI'] = 所得物價指數資料.loc[所得物價指數資料['所得層級別'] == '可支配所得中間60%家庭']['原始值[統計數值]'].reset_index(drop=True)
生育率['可支配所得最高20%家庭CPI'] = 所得物價指數資料.loc[所得物價指數資料['所得層級別'] == '可支配所得最高20%家庭']['原始值[統計數值]'].reset_index(drop=True)
生育率['房價所得比'] = 薪資.loc[3:12, '房價所得比[第4季][倍]'].reset_index(drop=True)
生育率['每人每月經常性薪資/總計[元]'] = 薪資.loc[3:12, '每人每月經常性薪資/總計[元]'].reset_index(drop=True)
生育率['經常性薪資中位數/總計[萬元]'] = 薪資.loc[3:12, '經常性薪資中位數/總計[萬元]'].reset_index(drop=True)
物價指數 = 物價指數.loc[物價指數['基本分類'] == '六.教養娛樂類'].reset_index(drop=True)
生育率['教養娛樂類CPI'] = 物價指數.loc[15:24, '原始值[統計數值]'].reset_index(drop=True)
印資料(生育率)
生育率.to_json('./時間資料有生育率.json', orient='index', force_ascii=False)

這份資料總共有 10 筆！


,統計期,育齡婦女一般生育率,育齡婦女總生育率,有偶婦女一般生育率,全體家庭CPI,可支配所得最低20%家庭CPI,可支配所得中間60%家庭CPI,可支配所得最高20%家庭CPI,房價所得比,每人每月經常性薪資/總計[元],經常性薪資中位數/總計[萬元],教養娛樂類CPI
0,102年,37.0,1205.0,83.0,93.53,92.39,93.42,93.98,15.01,37552.0,3.7,98.44
1,103年,42.0,1350.0,93.0,95.1,94.04,95.04,95.6,15.73,38218.0,3.8,99.43
2,104年,42.0,1340.0,93.0,95.1,94.45,95.05,95.51,15.75,38712.0,3.9,98.83
3,105年,41.0,1310.0,90.0,96.31,95.91,96.25,96.5,15.18,39213.0,3.9,98.82
4,106年,37.0,1215.0,83.0,96.89,96.55,96.86,97.07,14.99,39928.0,4.0,98.53
5,107年,35.0,1125.0,77.0,97.85,97.55,97.83,97.97,13.86,40959.0,4.1,99.50
6,108年,33.0,1085.0,73.0,98.66,98.4,98.61,98.83,13.94,41776.0,4.1,100.74
7,109年,30.0,985.0,67.0,98.48,98.45,98.45,98.55,15.78,42394.0,4.2,100.05
8,110年,29.0,960.0,66.0,100,100,100,100,16.29,43209.0,4.2,100.00
9,111年,25.0,845.0,58.0,102.69,102.62,102.66,102.75,15.77,44416.0,...,103.19
